In [ ]:
!pip install google-genai pydantic

In [2]:
import os
os.environ['GEMINI_API_KEY'] = 'MY_API_KEY'

In [3]:
import os
from google import genai
from pydantic import BaseModel, Field
from enum import Enum

# Initialize the Google GenAI client (uses GEMINI_API_KEY from environment)
client = genai.Client()

# Set the model for our agents
MODEL_ID = 'gemini-2.5-flash'

# --- 1. Define the Structured Schema for the Coordinator's Routing Decision ---
class IntentEnum(str, Enum):
    ORDER_STATUS = "order_status"
    PRODUCT_RETURN = "product_return"
    REFUND_REQUEST = "refund_request"
    UNKNOWN = "unknown"

class RouteDecision(BaseModel):
    intent: IntentEnum = Field(description="The categorized intent of the user's request.")
    extracted_details: str = Field(description="Any key entities extracted from the prompt, like order numbers or item names.")

# --- 2. Define the Specialized Subagents ---

def order_status_agent(details: str) -> str:
    """Specialized Subagent A: Handles order tracking."""
    print(f"   -> [Task A: Order Status Agent] Looking up: {details}")
    # In a real app, this agent might query a database or API here.
    return f"System Record: Order {details} shipped yesterday and is out for delivery."

def product_return_agent(details: str) -> str:
    """Specialized Subagent B: Handles return labels."""
    print(f"   -> [Task B: Product Return Agent] Processing return for: {details}")
    return f"System Record: A return shipping label for {details} has been generated."

def refund_request_agent(details: str) -> str:
    """Specialized Subagent C: Handles financial refunds."""
    print(f"   -> [Task C: Refund Agent] Processing refund for: {details}")
    return f"System Record: Refund initiated for {details}. It will process in 3-5 days."

# --- 3. Define the Coordinator Agent ---

def coordinator_agent(user_prompt: str) -> str:
    """
    The central coordinator agent.
    It uses an LLM to analyze the request, dynamically routes it,
    and synthesizes the final response.
    """
    print("-> [Coordinator] Analyzing request to dynamically route...\n")

    # Phase 1: AI Model Orchestration (Routing)
    routing_prompt = f"""Analyze the user's customer service request and determine the intent.
    Extract any relevant order numbers or items.

    User Request: {user_prompt}"""

    route_response = client.models.generate_content(
        model=MODEL_ID,
        contents=routing_prompt,
        config={
            'response_mime_type': 'application/json',
            'response_schema': RouteDecision,
            'temperature': 0.1,
        },
    )
    decision = route_response.parsed
    print(f"-> [Coordinator Decision] Route: {decision.intent.value.upper()} | Details: {decision.extracted_details}\n")

    # Phase 2: Dispatch to the appropriate Specialized Agent
    intermediate_output = ""
    if decision.intent == IntentEnum.ORDER_STATUS:
        intermediate_output = order_status_agent(decision.extracted_details)
    elif decision.intent == IntentEnum.PRODUCT_RETURN:
        intermediate_output = product_return_agent(decision.extracted_details)
    elif decision.intent == IntentEnum.REFUND_REQUEST:
        intermediate_output = refund_request_agent(decision.extracted_details)
    else:
        intermediate_output = "System Record: Could not determine how to route this request."

    print("\n-> [Coordinator] Intermediate output received. Synthesizing final response...")

    # Phase 3: Final Synthesis
    # The intermediate output is sent back to the Coordinator to format a user-friendly reply
    synthesis_prompt = f"""Write a polite, professional customer service reply based on the system output.

    User Request: {user_prompt}
    System Output: {intermediate_output}
    """
    final_response = client.models.generate_content(model=MODEL_ID, contents=synthesis_prompt)

    return final_response.text

# --- Execution ---
if __name__ == "__main__":
    # Example user request
    request = "Hi, I need to know where my package is. My order number is #XYZ-98765. It's taking forever."

    # Trigger the workflow
    final_output = coordinator_agent(request)

    print("\n=== Final Response to User ===")
    print(final_output)

-> [Coordinator] Analyzing request to dynamically route...

-> [Coordinator Decision] Route: ORDER_STATUS | Details: order number #XYZ-98765

   -> [Task A: Order Status Agent] Looking up: order number #XYZ-98765

-> [Coordinator] Intermediate output received. Synthesizing final response...

=== Final Response to User ===
Hello!

Thanks for reaching out about your order, #XYZ-98765. I understand you're eager to receive your package!

Good news! I've checked the status for you, and it was shipped yesterday and is currently **out for delivery** today. You should be receiving it very soon!

Please let us know if you have any other questions.
